In [1]:
import pandas as pd
import numpy as np
import math

from sklearn.feature_extraction.text import TfidfVectorizer

from nltk.corpus import stopwords

In [2]:
unida = pd.read_json('C:/Users/enterprise/Google Drive/fake_news_resources/raw_data/nacion_unida.json')
verif = pd.read_json('C:/Users/enterprise/Google Drive/fake_news_resources/raw_data/verificado_2018.json')

In [3]:
unida['text_list'] = unida['text'].apply(lambda x: str(x).split(' '))
unida['filter_text'] = unida['text_list'].apply(lambda x: [word for word in x if word not in stopwords.words('spanish')])

In [4]:
verif['text_list'] = verif['text'].apply(lambda x: str(x).split(' '))
verif['filter_text'] = verif['text_list'].apply(lambda x: [word for word in x if word not in stopwords.words('spanish')])

In [5]:
unida['filter_text_join'] = unida['filter_text'].apply(lambda x: ' '.join(x))
verif['filter_text_join'] = verif['filter_text'].apply(lambda x: ' '.join(x))

In [6]:
unida_str = unida['filter_text_join'].tolist()
verif_str = verif['filter_text_join'].tolist()

In [7]:
corpus = unida_str + verif_str

In [8]:
vectorizer = TfidfVectorizer()

In [9]:
matrix = vectorizer.fit_transform(corpus)

In [10]:
names = vectorizer.get_feature_names()

In [11]:
mat_uni = vectorizer.transform(unida_str)

In [12]:
mat_ver = vectorizer.transform(verif_str)

In [13]:
df_mat_uni = pd.DataFrame(mat_uni.todense(), columns=names)

In [14]:
df_mat_ver = pd.DataFrame(mat_ver.todense(), columns=names)

In [15]:
lst_mat_uni = df_mat_uni.values.tolist()
lst_mat_ver = df_mat_ver.values.tolist()

In [16]:
def simple_dot(a, b):
    dsum = 0
    for ((idx,), val) in np.ndenumerate(a):
        dsum += float(val) * float(b[idx])
    return(dsum)

In [17]:
def l2_norm(a):
    return(math.sqrt(np.dot(a, a)))

In [18]:
def cosine_similarity(a, b):
    return(np.dot(a,b) / (l2_norm(a) * l2_norm(b)))

In [19]:
maybe = []

for i in range(len(lst_mat_uni)):
    sim = cosine_similarity(np.array(lst_mat_uni[i]), np.array(lst_mat_ver[0]))
    maybe.append(sim)

In [20]:
m_maybe = max(maybe)
print(m_maybe)

0.10413711228219918


In [21]:
index = [i for i, j in enumerate(maybe) if j == m_maybe]

In [22]:
print(index)

[78]


In [24]:
print(verif['filter_text'][0])
print(unida['filter_text'][78])

['', 'La', 'nota', '“Funciona', 'boicot:', 'Se', 'derrumban', 'acciones', 'empresas', 'llamaban', 'votar', 'AMLO”', 'suma', '48', 'mil', 'interacciones', 'Facebook', 'sido', 'publicada', '141', 'veces', 'distintas', 'páginas', 'red', 'social.', 'Aunque', 'datos', 'ciertos,', 'información', 'descontextualizada.', 'Verificado', '2018', 'consultó', 'expertos', 'encontró', 'cambios', 'cotización', 'empresas', 'bolsa', 'responden', 'diversos', 'factores,', 'allá', 'elecciones.', 'Acciones', 'picada', 'La', 'nota', '“Funciona', 'boicot:', 'Se', 'derrumban', 'acciones', 'empresas', 'llamaban', 'votar', 'AMLO”,', 'publicada', 'portal', 'Nación', 'Unida,', 'explica', 'consecuencia', 'comunicados', 'empresarios', 'Germán', 'Larrea', 'Mota', '(Grupo', 'México)', 'y,', 'Héctor', 'Hernández', 'Pons', 'Torres', '(Grupo', 'Herdez),', 'modelo', 'político', 'encabeza', 'candidato', 'presidencial', 'Andrés', 'Manuel', 'López', 'Obrador,', 'acciones', 'empresas', 'perdieron', 'valor', 'bolsa.', 'Además',

In [25]:
print(cosine_similarity(np.array(lst_mat_uni[78]), np.array(lst_mat_ver[0])))

0.10413711228219918


In [27]:
print(verif['text'][0])
print(unida['text'][78])

 La nota “Funciona el boicot: Se derrumban acciones de empresas que llamaban a no votar por AMLO” suma 48 mil interacciones en Facebook y ha sido publicada 141 veces en distintas páginas de esa red social. Aunque hay datos ciertos, la información está descontextualizada. Verificado 2018 consultó con expertos y encontró que los cambios en la cotización de empresas en la bolsa responden a diversos factores, más allá de las elecciones. Acciones en picada La nota “Funciona el boicot: Se derrumban acciones de empresas que llamaban a no votar por AMLO”, publicada por el portal Nación Unida, explica que a consecuencia de los comunicados de los empresarios Germán Larrea Mota (Grupo México) y, Héctor Hernández Pons Torres (Grupo Herdez), en contra del modelo político que encabeza el candidato presidencial Andrés Manuel López Obrador, las acciones de estas empresas perdieron valor en la bolsa. Además de las anteriores se menciona a Grupo Lala; sin embargo, el posicionamiento de Lala ya fue desme